# <u>Data, Metadata and APIs</u>

## <u>Part 5: Maps and Open Data</u>

Now that you've extracted GPS coordinates from JPEG metadata and mapped it, you might be wondering what else you can map out. The short answer is... a lot. 

In this notebook, you'll see how to combine your knowledge of creating maps with your knowledge of data analysis with Pandas.

### <u>Find an Open Data Set that contains Location Data</u>

Here's a data set that tracks the location of all potholes filled by the City of Chicago for the past 7 days. Chicago is [known for its potholes](https://www.wbez.org/shows/curious-city/city-of-big-potholes-is-asphalt-the-best-choice-for-chicagos-streets/8bbd9e7a-b27e-4e00-a868-aa0b826b53b2), so this should be good. 

We will load this _.csv_ file in from a URL so that it is guaranteed to be the most up-to-date as possible:

In [1]:
# Note: the spike in traffic from Fremd may get us IP-banned by Chicago's Open Data portal.
#       If this happens, your teacher will share a static copy of Potholes_Patched.csv,
#       and you'll need to run the code "potholes_DF = pd.read_csv('Potholes_Patched.csv')"

import pandas as pd

potholes_DF = pd.read_csv("Potholes_Patched.csv")

# display the 3 most recent potholes that were filled
potholes_DF[-3:]

,ADDRESS,REQUEST DATE,COMPLETION DATE,NUMBER OF POTHOLES FILLED ON BLOCK,LATITUDE,LONGITUDE,LOCATION
106304,600 W 59TH ST,4/4/2022 13:51,4/4/2022 13:52,23,41.787195,-87.640291,POINT (-87.640291146857 41.787194823791)
106305,1000 W 59TH ST,4/4/2022 13:56,4/4/2022 13:57,10,41.787045,-87.649957,POINT (-87.649957148437 41.787044902409)
106306,6328 N LINCOLN AVE,3/31/2022 17:21,4/4/2022 12:14,1,41.996206,-87.717424,POINT (-87.717424109667 41.996205985167)


Check how many potholes were filled in the last week since the spreadsheet was generated:

In [2]:
print(len(potholes_DF))

106307


That's a lot of potholes. Now extract the location data, clean out the "nan" values, and store it as a list of tuples:

In [3]:
import numpy as np

lat = list(potholes_DF["LATITUDE"])

lon = list(potholes_DF["LONGITUDE"])

tuple_list = []

for i in range(len(lat)):
    coord = (lat[i],lon[i])
    tuple_list.append(coord)

tuple_list = [x for x in tuple_list if not np.isnan(x[1])]

Let's compare the length of *potholes_DF* to *tuple_list* to see how many "nan" values we cleaned out:

In [4]:
print(len(potholes_DF),len(tuple_list))

106307 105955


Depending on the week, there may be a handful of "nan" values to clean out. If you were lucky, there were none.

Now let's look at a few of the tuples in the list:

In [5]:
tuple_list[-10:]

[(41.7456901, -87.60546472),
 (41.79259436, -87.79514811),
 (41.96039461, -87.68338403),
 (41.78973533, -87.70494202),
 (41.81948525, -87.69375105),
 (41.99997707, -87.69576195),
 (41.73013202, -87.54693733),
 (41.78719482, -87.64029115),
 (41.7870449, -87.64995715),
 (41.99620599, -87.71742411)]

### <u>Maps with Markers</u>

Let's put a marker every place we found a pothole.
#### *WARNING: Adding more than 500 marker points could potentially crash your kernel!  To combat this, we are creating a list of 500 random entries from the original tuple_list.*

In [6]:
import numpy as np

tuple_list_500 = []
indicies_used = []
for i in range(500):                                # Loop 500 times
    random = np.random.randint(0,500)               # Generate random index number
    if random not in indicies_used:                 # Check if number has already been generated
        indicies_used.append(random)                # Add new number to list of used numbers
        tuple_list_500.append(tuple_list[random])   # Add the tuple from that index to the new list of 500
print(tuple_list_500[:10])

[(41.95949976, -87.73597143), (41.850011, -87.714006), (41.8131838, -87.64296852), (41.72877993, -87.63011264), (41.8125237, -87.63346773), (41.78778554, -87.66931624), (41.77878894, -87.70209746), (41.98619305, -87.74705899), (41.7333833, -87.5850216), (41.805791, -87.61387265)]


In [7]:
import os
# First, import folium package
import folium
from geopy.geocoders import Nominatim as NT

geo_locator = NT(user_agent = "geoapiExercises")

# Create a map with the starting location being the first element from your tuple list
user_map1 = folium.Map(location = [tuple_list_500[0][0], tuple_list_500[0][1]], zoom_start = 17)

# Create a marker/pin for each point on the map where there is a pothole
for lon, lat in tuple_list_500:
    folium.Marker(location = [lon, lat], popup = folium.Popup("Pothole here!", parse_html = True, max_width = 100)).add_to(user_map1)

user_map1

**<u>Question 1:</u>** Look at the marker map at various zoom levels. What do you notice above the graph? Comment on anything interesting you see and try to summarize "the good" and "the bad" in this visualization.

**<u>Your Answer: It at least gives exact locations and also where the owner took it, alongside this, they can ovrlap each other, making it nicer. However, I find the markers to be very large, and I get that you are supposed to scale up with the zoom, it would be better if it was just a tad bit smaller.</u>** 

### <u>Create a Heatmap</u>

Instead of markers, let's make a heat map:
#### *WARNING: Adding more than 500 marker points could potentially crash your kernel!  To combat this, we are again using the list of 500 random entries from the original tuple_list.*

In [8]:
from folium import plugins
from folium.plugins import HeatMap

heat_map = folium.Map(location = [tuple_list_500[0][0], tuple_list_500[0][1]], zoom_start = 14)

HeatMap(tuple_list_500).add_to(heat_map)

heat_map

**<u>Question 2:</u>** Look at the heatmap at various zoom levels. What do you notice above the graph? Comment on anything interesting you see and try to summarize "the good" and "the bad" in this visualization.

**<u>Your Answer: Also a good area, if you are trying to search. Useful for police investigations. However it only gives a general area, not the EXACT point.</u>** 

### <u>Task 1: Find your own dataset!</u>

You are going to create a marker map **and** a heatmap from a dataset you have found. For Task 1, find a dataset with location data (GPS coordinates!). Fill in the following:

_Name:_ Black Bear GPS

_Date:_ 12/31/1979

_Source for Data Set:_ I found it in my folder

_URL for Data Set:_ NONE

_Description of Data Set:_ This dataset is a set of data pertaining to bears location. However, this was onlt targeting the specific bears. the BLACK bears. Now, the BLACK bears are the ones being found, not the other colored bears. only the BLACK bears. 

_File Format for Data Set:_ .csv

_Age of Data Set:_ 44 year old middle age guy

### <u>Task 2: Show some entries fom your dataset</u>

Import your data set as a Pandas Data Frame, then show the last 10 entries:

In [9]:
# Your code here
import pandas as pd


Bears_DF = pd.read_csv("Black Bear GPS.csv")

Bears_DF[-10:]

,FID,AID,Year,Julianday,Hour,Minute,Activity,Temperatur,Latitude,Longitude,HDOP,NumSats,FixTime,2D_3D,POINT_X,POINT_Y
19595,19595,31412,2012,158,16,0,57,19,58.52109,-122.00019,2.5,4,32,3,558235.648843,6.487160e+06
19596,19596,31412,2012,158,16,30,65,19,58.52232,-122.00092,3.8,4,27,3,558191.094016,6.487297e+06
19597,19597,31412,2012,158,17,2,2,27,58.52231,-122.00123,2.4,5,117,3,558173.055550,6.487295e+06
19598,19598,31412,2012,158,17,30,25,24,58.52209,-122.00167,4.2,5,22,3,558147.793075,6.487271e+06
19599,19599,31412,2012,158,18,0,65,19,58.52322,-122.00208,2.3,5,36,3,558122.044751,6.487396e+06
19600,19600,31412,2012,158,18,30,71,19,58.52222,-122.00334,2.2,5,37,3,558050.313542,6.487284e+06
19601,19601,31412,2012,158,19,1,71,18,58.52102,-122.00199,1.6,6,95,3,558130.925359,6.487151e+06
19602,19602,31412,2012,158,19,31,70,19,58.52002,-122.00092,2.0,6,48,3,558194.902425,6.487041e+06
19603,19603,31412,2012,158,20,1,68,18,58.52254,-122.00408,3.8,4,53,3,558006.686128,6.487319e+06
19604,19604,31412,2012,158,20,32,75,18,58.52508,-122.00963,2.0,4,120,2,557679.275543,6.487597e+06


### <u>Task 3: Create a list of tuples</u>

Use your dataset to create a list of tuples (a list of DD coordinates) representing the locations in your dataset:
#### *WARNING: Adding more than 500 marker points could potentially crash your kernel!  To combat this, create a list of 500 random entries from the original list of tuples.*

In [10]:
# Your code here
import numpy as np

latMK2 = list(Bears_DF["Latitude"])

lonMK2 = list(Bears_DF["Longitude"])

tuple_listMK2 = []

for i in range(len(latMK2)):
    coord = (latMK2[i],lonMK2[i])
    tuple_listMK2.append(coord)

tuple_listMK2 = [x for x in tuple_listMK2 if not np.isnan(x[1])]

### <u>Task 4: Create a marker map from your data</u>

Create a marker map using your list of tuples from above.

In [11]:
# Your code here
import numpy as np

tuple_list_500MK2 = []
indicies_usedMK2 = []
for i in range(500):                                # Loop 500 times
    random = np.random.randint(0,500)               # Generate random index number
    if random not in indicies_usedMK2:                 # Check if number has already been generated
        indicies_usedMK2.append(random)                # Add new number to list of used numbers
        tuple_list_500MK2.append(tuple_listMK2[random])   # Add the tuple from that index to the new list of 500
print(tuple_list_500MK2[:10])



import os
# First, import folium package
import folium
from geopy.geocoders import Nominatim as NT

geo_locator = NT(user_agent = "geoapiExercises")

# Create a map with the starting location being the first element from your tuple list
user_map2 = folium.Map(location = [tuple_list_500MK2[0][0], tuple_list_500MK2[0][1]], zoom_start = 17)

# Create a marker/pin for each point on the map where there is a pothole
for lonMK2, latMK2 in tuple_list_500MK2:
    folium.Marker(location = [lonMK2, latMK2], popup = folium.Popup("Goofy ahh bear here!", parse_html = True, max_width = 100)).add_to(user_map2)

user_map2

[(59.34285, -122.20964), (59.30476, -122.20869), (59.34817, -122.15322), (59.33007, -122.19889), (59.37357, -122.1968), (59.33192, -122.24154), (59.38626, -122.15315), (59.3675, -122.24128), (59.27537, -122.13348), (59.31927, -122.27632)]


### <u>Task 5: Create a heatmap from your data</u>

Create a **heatmap** using your list of tuples from above.

In [12]:
# Your code here
from folium import plugins
from folium.plugins import HeatMap

heat_map2 = folium.Map(location = [tuple_list_500MK2[0][0], tuple_list_500MK2[0][1]], zoom_start = 14)

HeatMap(tuple_list_500MK2).add_to(heat_map2)

heat_map2

### <u>Task 6: Comment on what you see</u>

Look at your marker map and your heatmap at various zoom levels. Comment on anything interesting or notable that you see. 

**<u>Your Answer: What is the most notable is the fact that most of this is  wilderness, not a city to be seen in sight, aside from the few rivers, there really isn't much going on. Another thing to note is that most of the bears are spread out, not much really travel together, and they seem to keep to themselves</u>** 

### <u>Task 7: Brainstorm further study</u>

If you had more time and resources, what else would you like to explore using the GPS data in this dataset?

**<u>Your Answer: I want to explore if certain situations would be able to bring them all to some central point that could be able  to bring them all together</u>**